# Medical Image - TFM
<h4>subtitle: Generación de una tubería distribuida para la extracción de características en imágenes médicas patológicas</h4>
license: European Union Public Licence (EUPL) v1.2

<table>
  <tr> <td> author name: </td> <td> Israel Llorens </td> </tr>
  <tr> <td> email: </td> <td> sanchezis@hotmail.com </td> </tr>
</table>

<h7>date: 2025/03/23</h7>

---

In [1]:
%load_ext autoreload
%autoreload

## Inicialización

In [1]:
import init
from digital_pathology.spark import spark

25/03/24 11:22:55 WARN Utils: Your hostname, Airon.local resolves to a loopback address: 127.0.0.1; using 192.168.100.210 instead (on interface en0)
25/03/24 11:22:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 11:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/24 11:22:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/24 11:22:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/24 11:22:56 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/24 11:22:56 INFO SharedState: Warehouse path is 'file:/Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/notebooks/spark-warehouse'.


In [10]:
import os
try:
    import pyspark
    from pyspark.sql.functions import col, isnan, when, count,to_date,year,month,expr,hour,dayofweek,lower,array_remove,collect_list,lit
    from pyspark.sql.functions import pandas_udf,split
    from pyspark.sql.types import StructField,StructType,StringType,DoubleType,FloatType,IntegerType, ArrayType, DoubleType, LongType
    import pyspark.sql.functions as F
except:
    pass
import pandas as pd

25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_5_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_2_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_4_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)


In [2]:
# Copyright (c) 2024 Israel Llorens
# Licensed under the EUPL-1.2  

__author__ = "Israel Llorens <sanchezis@hotmail.com>"
__copyright__ = "Copyright 2024, Israel Llorens"
__license__ = "EUPL-1.2"

import logging
import os

import pandas as pd
import glob
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

from tiatoolbox import logger
from tiatoolbox.models.architecture.unet import UNetModel
from tiatoolbox.models.engine.semantic_segmentor import (
    IOSegmentorConfig,
    SemanticSegmentor,
)
from tiatoolbox.utils.misc import download_data, imread
from tiatoolbox.utils.visualization import overlay_prediction_mask
from tiatoolbox.wsicore.wsireader import WSIReader

try:
    import pyspark
    from pyspark.sql.functions import col, isnan, when, count,to_date,year,month,expr,hour,dayofweek,lower,array_remove,collect_list,lit
    from pyspark.sql.functions import pandas_udf,split
    from pyspark.sql.types import ArrayType, DoubleType, StringType
    from pyspark.sql.types import StructField,StructType,StringType,DoubleType,FloatType,IntegerType, LongType
    import pyspark.sql.functions as F
except:
    pass


# Clear logger to use tiatoolbox.logger
import logging
import warnings

if logging.getLogger().hasHandlers():
    logging.getLogger().handlers.clear()

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm

from tiatoolbox import logger
from tiatoolbox.models.architecture.unet import UNetModel
from tiatoolbox.models.engine.semantic_segmentor import (
    IOSegmentorConfig,
    SemanticSegmentor,
)
from tiatoolbox.utils.misc import download_data, imread
from tiatoolbox.utils.visualization import overlay_prediction_mask
from tiatoolbox.wsicore.wsireader import WSIReader

from urllib import request
import certifi
import ssl
import os

import numpy as np
import histomicstk as htk
import skimage
import scipy as sp

import init
from digital_pathology.spark import spark
from digital_pathology.process.model_selection import ResNetModel

|2025-03-24|14:16:14.191| [WARNING] /Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

|2025-03-24|14:16:18.578| [WARNING] /Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/.venv/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()

INFO:root:****************************CREATING SPARK CLUSTER*******************************
25/03/24 14:16:19 WARN Utils: Your hostname, Airon.local resolves to a loopback address: 127.0.0.1; using 192.168.100.210 inst

## Gold Data

In [7]:
glob.glob(os.path.join('..','data', '*.parquet') )

['../data/3-nucleotids.parquet',
 '../data/2-stardist-nucleotids.parquet',
 '../data/0-extract.parquet',
 '../data/1-download.parquet',
 '../data/4-nucleotids.parquet']

In [8]:
data_0 = spark.read.parquet(os.path.join('..','data', '0-extract.parquet'))
data_1 = spark.read.parquet('../data/1-download.parquet')
data_3 = spark.read.parquet('../data/3-nucleotids.parquet')
data_4 = spark.read.parquet('../data/4-nucleotids.parquet')

25/03/24 14:20:21 INFO InMemoryFileIndex: It took 3 ms to list leaf files for 1 paths.
25/03/24 14:20:21 INFO SparkContext: Starting job: parquet at NativeMethodAccessorImpl.java:0
25/03/24 14:20:21 INFO DAGScheduler: Got job 4 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
25/03/24 14:20:21 INFO DAGScheduler: Final stage: ResultStage 4 (parquet at NativeMethodAccessorImpl.java:0)
25/03/24 14:20:21 INFO DAGScheduler: Parents of final stage: List()
25/03/24 14:20:21 INFO DAGScheduler: Missing parents: List()
25/03/24 14:20:21 INFO DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[13] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
25/03/24 14:20:21 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 106.2 KiB, free 9.4 GiB)
25/03/24 14:20:21 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 38.5 KiB, free 9.4 GiB)
25/03/24 14:20:21 INFO BlockManagerInfo: Added broadca

In [15]:
data_4.show(truncate=False)

+---------------------------------------------------------------+----------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|filename                                                       |patient_key           |patient_id |extract_tumor(filename, data/patient_extracts)                                                                                                        |
+---------------------------------------------------------------+----------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|patient_000_node_1.tif_tile_44_x0_y166912_score22950.0.png     |patient_000_node_1.tif|patient_000|{[ 28738  32662      0      0 987176], [('Tumour', 2.7407), ('Stroma', 3.1149), ('Inflamatory', 0.0), ('Necrosis', 0.0), ('Others', 94.1444)]}  

25/03/24 14:22:38 INFO FileSourceStrategy: Pushed Filters: 
25/03/24 14:22:38 INFO FileSourceStrategy: Post-Scan Filters: 
25/03/24 14:22:38 INFO CodeGenerator: Code generated in 11.724084 ms
25/03/24 14:22:38 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 220.0 KiB, free 9.4 GiB)
25/03/24 14:22:38 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 36.7 KiB, free 9.4 GiB)
25/03/24 14:22:38 INFO BlockManagerInfo: Added broadcast_21_piece0 in memory on 192.168.100.210:61924 (size: 36.7 KiB, free: 9.4 GiB)
25/03/24 14:22:38 INFO SparkContext: Created broadcast 21 from showString at NativeMethodAccessorImpl.java:0
25/03/24 14:22:38 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
25/03/24 14:22:38 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
25/03/24 14:22:38 INFO DAGScheduler: Got job 14 (showString at Nat